In [1]:
import re
import tkinter as tk
from tkinter import scrolledtext, messagebox, ttk

# Token types
TOKEN_TYPES = [
    ('KEYWORD', r'\bfunction\b|\breturn\b|\bif\b|\belse\b|\bfor\b|\bwhile\b|\bvar\b|\blet\b|\bconst\b'),
    ('ARROW', r'=>'),
    ('IDENTIFIER', r'[a-zA-Z_$][a-zA-Z_$0-9]*'),
    ('NUMBER', r'\b\d+(\.\d+)?\b'),
    ('OPERATOR', r'[+\-*/<>!&|%^~]'),
    ('PUNCTUATION', r'[(){},;]'),
    ('ASSIGNMENT', r'='),
    ('STRING', r'"[^"\\]*(\\.[^"\\]*)*"|\'[^\'\\]*(\\.[^\'\\]*)*\''),
    ('WHITESPACE', r'\s+'),
    ('COMMENT', r'//.*?$|/\*.*?\*/'),
]

# Identifier naming convention
def is_valid_identifier_name(name):
    return re.match(r'^[a-zA-Z_$][a-zA-Z_$0-9]*$', name) is not None

# Lexer
def tokenize(code):
    tokens = []
    while code:
        matched = False
        for token_type, pattern in TOKEN_TYPES:
            regex = re.compile(pattern)
            match = regex.match(code)
            if match:
                matched = True
                text = match.group(0)
                if token_type != 'WHITESPACE' and token_type != 'COMMENT':
                    tokens.append((token_type, text))
                code = code[len(text):]
                break
        if not matched:
            raise SyntaxError(f'Unexpected character: {code[0]}')
    return tokens

class Parser:
    def __init__(self, tokens):
        self.tokens = tokens
        self.position = 0

    def current_token(self):
        return self.tokens[self.position] if self.position < len(self.tokens) else (None, None)

    def next_token(self):
        self.position += 1
        return self.current_token()

    def expect(self, token_type, value=None):
        token = self.current_token()
        if token[0] != token_type or (value and token[1] != value):
            raise SyntaxError(f'Expected {token_type} {value}, got {token}')
        self.next_token()

    def parse(self):
        if self.current_token()[1] == 'function':
            return self.parse_function_declaration()
        elif self.current_token()[0] == 'KEYWORD' and self.current_token()[1] in ('const', 'let', 'var'):
            return self.parse_variable_declaration()
        else:
            raise SyntaxError(f'Unexpected token: {self.current_token()}')

    def parse_function_declaration(self):
        self.expect('KEYWORD', 'function')
        identifier = self.current_token()
        self.expect('IDENTIFIER')
        self.expect('PUNCTUATION', '(')
        parameters = self.parse_parameters()
        self.expect('PUNCTUATION', ')')
        self.expect('PUNCTUATION', '{')
        body, has_return = self.parse_statements()
        self.expect('PUNCTUATION', '}')
        if not has_return:
            raise SyntaxError('Function must return something.')
        return {'type': 'FunctionDeclaration', 'name': identifier[1], 'params': parameters, 'body': body}

    def parse_variable_declaration(self):
        keyword = self.current_token()
        self.expect('KEYWORD')
        identifier = self.current_token()
        self.expect('IDENTIFIER')
        if self.current_token()[0] == 'ASSIGNMENT':
            self.expect('ASSIGNMENT')
            if self.current_token()[1] == '(':
                return self.parse_arrow_function_declaration(keyword, identifier)
            else:
                value = self.current_token()
                self.next_token()  # Move past the value token
                return {'type': 'VariableDeclaration', 'keyword': keyword[1], 'name': identifier[1], 'value': value[1]}
        else:
            raise SyntaxError(f'Expected assignment after variable declaration')

    def parse_arrow_function_declaration(self, keyword, identifier):
        self.expect('PUNCTUATION', '(')
        parameters = self.parse_parameters()
        self.expect('PUNCTUATION', ')')
        self.expect('ARROW', '=>')
        self.expect('PUNCTUATION', '{')
        body, has_return = self.parse_statements()
        self.expect('PUNCTUATION', '}')
        if not has_return:
            raise SyntaxError('Arrow function must return something.')
        return {'type': 'ArrowFunctionDeclaration', 'keyword': keyword[1], 'name': identifier[1], 'params': parameters, 'body': body}

    def parse_statements(self):
        statements = []
        has_return = False
        while self.current_token()[0] != 'PUNCTUATION' or self.current_token()[1] != '}':
            token = self.current_token()
            if token[0] == 'KEYWORD' and token[1] == 'return':
                self.expect('KEYWORD', 'return')
                value = self.current_token()
                self.next_token()
                self.expect('PUNCTUATION', ';')
                statements.append({'type': 'ReturnStatement', 'value': value[1]})
                has_return = True
            else:
                raise SyntaxError(f'Unexpected token in function body: {token}')
        return statements, has_return
    def parse_parameters(self):
        parameters = []
        while self.current_token()[0] != 'PUNCTUATION' or self.current_token()[1] != ')':
            if self.current_token()[0] == 'IDENTIFIER':
                parameters.append(self.current_token()[1])
                self.next_token()
            if self.current_token()[0] == 'PUNCTUATION' and self.current_token()[1] == ',':
                self.expect('PUNCTUATION', ',')
        return parameters

def analyze_code():
    code = code_input.get("1.0", tk.END)
    print("Code Input:", code)
    try:
        tokens = tokenize(code)
        print("Tokens:", tokens)
        display_tokens(tokens)
        parser = Parser(tokens)
        ast = parser.parse()
        print("AST:", ast)
        display_ast(ast)
    except SyntaxError as e:
        print(f"Syntax Error: {e}")
        messagebox.showerror("Syntax Error", str(e))
    except Exception as e:
        print(f"Error: {e}")
        messagebox.showerror("Error", str(e))

def display_tokens(tokens):
    for item in tokens_tree.get_children():
        tokens_tree.delete(item)
    for token in tokens:
        tokens_tree.insert("", "end", values=token)

def display_ast(ast, parent=""):
    if isinstance(ast, dict):
        node_id = ast_tree.insert(parent, "end", text=ast.get('type'))
        for key, value in ast.items():
            if key != 'type':
                child_id = ast_tree.insert(node_id, "end", text=key)
                display_ast(value, child_id)
    elif isinstance(ast, list):
        for item in ast:
            display_ast(item, parent)
    else:
        ast_tree.insert(parent, "end", text=str(ast))



root = tk.Tk()
root.title("JavaScript Function Analyzer")
root.geometry("800x600")

tk.Label(root, text="Enter JavaScript Code:").pack()

code_input = scrolledtext.ScrolledText(root, width=80, height=10)
code_input.pack()

analyze_button = tk.Button(root, text="Analyze", command=analyze_code)
analyze_button.pack()

tk.Label(root, text="Tokens:").pack()

tokens_frame = tk.Frame(root)
tokens_frame.pack()

tokens_tree = ttk.Treeview(tokens_frame, columns=("Type", "Value"), show="headings")
tokens_tree.heading("Type", text="Type")
tokens_tree.heading("Value", text="Value")
tokens_tree.pack(side="left", fill="y")

tokens_scroll = ttk.Scrollbar(tokens_frame, orient="vertical", command=tokens_tree.yview)
tokens_scroll.pack(side="right", fill="y")
tokens_tree.configure(yscroll=tokens_scroll.set)

tk.Label(root, text="Abstract Syntax Tree (AST):").pack()

ast_frame = tk.Frame(root)
ast_frame.pack(fill="both", expand=True)

ast_tree = ttk.Treeview(ast_frame)
ast_tree.pack(side="left", fill="both", expand=True)

ast_scroll = ttk.Scrollbar(ast_frame, orient="vertical", command=ast_tree.yview)
ast_scroll.pack(side="right", fill="y")
ast_tree.configure(yscroll=ast_scroll.set)

root.mainloop()


Code Input: function abc(){
	return 2;
}

Tokens: [('KEYWORD', 'function'), ('IDENTIFIER', 'abc'), ('PUNCTUATION', '('), ('PUNCTUATION', ')'), ('PUNCTUATION', '{'), ('KEYWORD', 'return'), ('NUMBER', '2'), ('PUNCTUATION', ';'), ('PUNCTUATION', '}')]
AST: {'type': 'FunctionDeclaration', 'name': 'abc', 'params': [], 'body': [{'type': 'ReturnStatement', 'value': '2'}]}
Code Input: function abc(){
	//a
	return 2;
}

Tokens: [('KEYWORD', 'function'), ('IDENTIFIER', 'abc'), ('PUNCTUATION', '('), ('PUNCTUATION', ')'), ('PUNCTUATION', '{'), ('OPERATOR', '/'), ('OPERATOR', '/'), ('IDENTIFIER', 'a'), ('KEYWORD', 'return'), ('NUMBER', '2'), ('PUNCTUATION', ';'), ('PUNCTUATION', '}')]
Syntax Error: Unexpected token in function body: ('OPERATOR', '/')
Code Input: function abc(){
	//a;
	return 2;
}

Tokens: [('KEYWORD', 'function'), ('IDENTIFIER', 'abc'), ('PUNCTUATION', '('), ('PUNCTUATION', ')'), ('PUNCTUATION', '{'), ('OPERATOR', '/'), ('OPERATOR', '/'), ('IDENTIFIER', 'a'), ('PUNCTUATION', ';'), 